<a href="https://colab.research.google.com/github/Sowmya74/Article_Lang_Converter/blob/main/NLP/Article_Language_Conversion/Article_Lang_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP Project**

##**Converting a News Article which is in English Language to Hindi Language using Newspaper3k**

### Firstly, to get the paper, we use the newspaper3k library to collect and sparse. After that using langdetect, we detect the language of the paper and then using the MarianMT model which is a transformer, we convert the article text to Hindi Language.

In [ ]:
#@title #Installing libraries
!pip install indic-nlp-library
!pip install newspaper3k transformers langdetect sentencepiece torch
!pip install sacrebleu rouge-score pyterrier nltk
!pip install sacrebleu pyterrier gradio
import gradio as gr
import nltk

In [ ]:
# Download the punkt tokenizer for sentence splitting
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from newspaper import Article

url = 'https://indiaai.gov.in/news/meta-ai-rolls-out-in-india-the-leading-ai-assistant-is-now-at-your-fingertip'
article = Article(url)

# Download and parse the article
article.download()
article.parse()

# Print the article text
print(article.text)

Meta AI works tremendously for users, from asking it in your WhatsApp group chat to recommending top restaurants to give ideas of places to stop on a road trip.

Meta has recently announced the long-awaited availability of Meta AI in India on WhatsApp, Instagram, Facebook Messenger and the meta.ai portal. You can now use Meta AI on these platforms to get things done, learn, create and connect with the things that matter to you. Built with Meta Llama 3, Meta AI is one of the world’s leading AI assistants, already on your phone, in your pocket for free across more than a dozen countries.

Meta first launched Meta AI at last year’s Connect. Their official statement said, “We’ve been bringing the latest version of Meta AI built with Llama 3 to people worldwide. Thanks to Meta Llama 3, Meta AI is smarter, faster and more fun than ever before.”

Meta AI works tremendously for users, from asking it in your WhatsApp group chat to recommending top restaurants to give ideas of places to stop on 

In [ ]:
from langdetect import detect

language = detect(article.text)
print(f'The language of the article is: {language}')

The language of the article is: en


In [ ]:
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
# Load the MarianMT model for English to Hindi translation
model_name_en_hi = 'Helsinki-NLP/opus-mt-en-hi'
tokenizer_en_hi = MarianTokenizer.from_pretrained(model_name_en_hi)
model_en_hi = MarianMTModel.from_pretrained(model_name_en_hi)

def translate_chunk(text, tokenizer, model):
    try:
        inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(**inputs)
        translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        return translated_text
    except Exception as e:
        print(f"Error translating chunk: {text}\nError: {e}")
        return ""

def translate_long_text(text, tokenizer, model):
    # Split text into sentences
    sentences = sent_tokenize(text)
    translated_sentences = [translate_chunk(sentence, tokenizer, model) for sentence in sentences]

    # Combine sentences into paragraphs
    # Adding paragraph breaks based on original structure
    # Assuming each sentence could be part of a paragraph
    return '\n'.join(translated_sentences)

# Translate the article from English to Hindi
translated_text_hi = translate_long_text(article.text, tokenizer_en_hi, model_en_hi)

# Print the translated text
print("Translated to Hindi:\n", translated_text_hi)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Translated to Hindi:
 मेटा एआई उपभोक्ताओं के लिए अत्यधिक रूप से काम करता है, अपने में यह पूछने से कि कौन सी आन्तरिक समूह गपशप शीर्ष होटलों के विचारों को एक सड़क यात्रा पर बंद करने के लिए सुझाव देने के लिए सुझाव दे रहा है.
मेटा ने हाल ही में भारत में मेटा एआई की लंबी मात्रा की घोषणा की है कि क्या हो रहा है, इंस्टाग्राम, फेसबुक, फेसबुक तथा मेटा. पोर्टल.
अब आप काम पूरा करने के लिए इन मंच पर मेटा एआई का उपयोग कर सकते हैं, सीख सकते हैं, और उन चीज़ों से कनेक्ट कर सकते हैं जो आपके लिए मायने रखती हैं.
मेटा लला 3 मेटा एआई के साथ बनाया गया है, जो कि दुनिया के प्रमुख एआई सहायकों में से एक है, पहले से ही आपके फोन पर, एक दर्जन से अधिक देशों में मुफ्त के लिए अपनी जेब में.
पिछले साल के कनेक्ट होने पर मेटा एआई को पहली बार चालू किया गया.
उनके सरकारी बयान ने कहा, “हम दुनिया भर के लोगों के लिए मेटा एआई का नवीनतम संस्करण तैयार कर रहे हैं ।
मेटा Lmaaa 3, मेटा एआई के लिए धन्यवाद, पहले से कहीं अधिक तेज़ है, और अधिक मज़ेदार है। मेटा एआई उपभोक्ताओं के लिए बहुत बढ़िया काम करता है, इसे पूछने से, क्या क्लीनर समूह

In [ ]:
# Save the translated text
output_file_path = 'translated_article.txt'
with open(output_file_path, 'w', encoding='utf-8') as f:
    f.write(translated_text_hi)

print(f"Translation complete. Check the '{output_file_path}' file for the translated article.")

Translation complete. Check the 'translated_article.txt' file for the translated article.


In [ ]:
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
def evaluate_translations(reference_text, translated_text):
    # BLEU Score
    bleu_score = corpus_bleu([translated_text], [[reference_text]])
    print(f"BLEU Score: {bleu_score.score}")

    # ROUGE Score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_text, translated_text)
    print(f"ROUGE Scores: {scores}")

reference_text = article.text
translated_text = translated_text_hi
evaluate_translations(article.text, translated_text_hi)

BLEU Score: 0.2672026478339213
ROUGE Scores: {'rouge1': Score(precision=0.6666666666666666, recall=0.004728132387706856, fmeasure=0.009389671361502348), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.6666666666666666, recall=0.004728132387706856, fmeasure=0.009389671361502348)}
TER Score: 0


## **English to Hindi Translator**

In [ ]:
def process_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()

        # Check language of the article
        language = detect(article.text)
        if language != 'en':
            return "The article is not in English. Detected language: {}".format(language)

        # Translate the article from English to Hindi
        translated_text_hi = translate_long_text(article.text, tokenizer_en_hi, model_en_hi)

        return translated_text_hi
    except Exception as e:
        return f"An error occurred: {e}"

# Create Gradio interface
iface = gr.Interface(
    fn=process_url,
    inputs=gr.Textbox(label="Enter URL"),
    outputs=gr.Textbox(label="Translated Text"),
    title="English to Hindi Article Translator",
    description="Enter the URL of an English article to translate it into Hindi."
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://41abaa2f46c14faeab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
